In [1]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config, sub_root_path
from bcnf.train import Trainer
from bcnf import CondRealNVP_v2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
MODEL_NAME = 'trajectory_FC_small'

In [4]:
config_path_pattern = os.path.join("{{BCNF_ROOT}}", "configs", "runs", f"{MODEL_NAME}.yaml")
config_path = sub_root_path(config_path_pattern)
print(f'Loading config from {config_path}')

config = load_config(config_path, verify=False)

Loading config from /home/psaegert/Projects/bcnf/configs/runs/trajectory_FC_small.yaml


In [5]:
model = CondRealNVP_v2.from_config(config).to(device)

print(f'{model.n_params:,}')

128,257


In [6]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/psaegert/Projects/bcnf/data/bcnf-data/fixed_data_render_2s_15FPS/train...


Loading data from directory:  50%|█████     | 2/4 [00:15<00:14,  7.33s/it, file=fixed_data_render_2s_15FPS_3.pkl]

In [ ]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: psaegert (balisticcnf). Use `wandb login --relogin` to force relogin


Train: -13.9519 - Val: -17.6715 (avg: -17.1558, min: -17.1006) | lr: 2.00e-04 - Patience: 2/500 - z: (-0.0419 ± 0.1796) ± (0.7029 ± 0.2037):   1%|          | 480/50000 [04:50<8:49:14,  1.56it/s]

In [ ]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path_pattern}, f)

print(f"Model saved to {get_dir('models', 'bcnf-models', MODEL_NAME)}")

Model saved to /home/psaegert/Projects/bcnf/models/bcnf-models/trajectory_FC_large
